In [11]:
import pandas as pd

train_df = pd.read_csv("house_data/train.csv")
test_df = pd.read_csv("house_data/test.csv")

train_df.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [12]:
X = train_df.drop("SalePrice", axis=1)
y = train_df["SalePrice"]


In [13]:
# Numeric features only
X = X.select_dtypes(include=["int64", "float64"])
test_df = test_df.select_dtypes(include=["int64", "float64"])

# Fill missing values with median
X = X.fillna(X.median())
test_df = test_df.fillna(test_df.median())


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)


In [16]:
import numpy as np
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("RMSE:", rmse)


RMSE: 36879.8193219275


In [17]:
rmse_log = np.sqrt(
    mean_squared_error(
        np.log1p(y_val),
        np.log1p(y_pred)
    )
)

print("Log RMSE:", rmse_log)


Log RMSE: 0.1893090830955901


In [18]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)

rmse_rf = np.sqrt(mean_squared_error(y_val, rf_pred))
print("RF RMSE:", rmse_rf)


RF RMSE: 28924.034354484746


In [19]:
rf.fit(X, y)
test_preds = rf.predict(test_df)


In [20]:
submission = pd.DataFrame({
    "Id": test_df.index + 1,
    "SalePrice": test_preds
})

submission.to_csv("house_data/submission.csv", index=False)
